<a href="https://colab.research.google.com/github/SomTu/RAD-2025/blob/main/code/01RAD_Ex12_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle house data set



## Downloading the Kaggle house rent dataset

The dataset we will use comes from Kaggle:

- *House Rent Prediction Dataset*  
  https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset/data

To download directly from Kaggle inside this notebook you need a Kaggle
API token (see *Account ? API ? Create New Token* on Kaggle). The cell
below assumes you have configured your `KAGGLE_USERNAME` and
`KAGGLE_KEY` environment variables or placed `kaggle.json` in the
standard location.

Exampe of Auto NB - let's beat it

https://www.kaggle.com/code/sahityasetu/boosting-algorithms-for-machine-learning


In [ ]:

# Download the Kaggle house rent dataset using kagglehub (no API key needed for public data)
try:
    import kagglehub  # lightweight helper for Kaggle datasets
except ImportError:  # pragma: no cover
    %pip install -q kagglehub
    import kagglehub

# Download latest version of the dataset; this returns a local directory path
path = kagglehub.dataset_download("iamsouravbanerjee/house-rent-prediction-dataset")
print("Path to dataset files:", path)


In [ ]:

from pathlib import Path
import pandas as pd

# `path` is a directory returned by kagglehub; locate the CSV inside it
dataset_dir = Path(path)
candidates = list(dataset_dir.rglob("House_Rent_Dataset.csv"))
if not candidates:
    raise FileNotFoundError(f"House_Rent_Dataset.csv not found under {dataset_dir}")

csv_path = candidates[0]
print("Loading data from:", csv_path)
house = pd.read_csv(csv_path)
print("Shape:", house.shape)
house_orig = house
house.head()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor



In [ ]:
print(list(house.columns))
print(house.describe(include='all'), "\n\n")
print(house.isna().sum())
sns.pairplot(data=house)

for col in house.columns:
    print(f"\nColumn: {col}")
    print(house[col].unique())


The columns are ['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type', 'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred', 'Bathroom', 'Point of Contact']

'Posted On' should not have any effect on Rent, therefore will not be used for the model.

'Area Type' has half as many unique values as there are total of samples. Author sees no sensible way to include them in the model.

Initial feature choice is ['BHK', 'Size', 'Floor', 'City', 'Furnishing Status', 'Bathroom']

In [ ]:


def add_floor_columns(
    df: pd.DataFrame,
    source_col: str,
    current_col: str = "current_floor",
    max_col: str = "max_floor",
    inplace: bool = True,
):
    """
    Parses floor information from a column and adds two new columns:
    current floor and max floor.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe
    source_col : str
        Column containing floor strings (e.g. '3 out of 10')
    current_col : str, optional
        Name of output column for current floor
    max_col : str, optional
        Name of output column for max floor
    inplace : bool, optional
        If True, modifies df in place. If False, returns a copy.

    Returns
    -------
    pd.DataFrame
        DataFrame with added columns
    """

    if not inplace:
        df = df.copy()

    floor_map = {
        "Ground": 0,
        "Upper Basement": -1,
        "Lower Basement": -2,
    }

    pattern = re.compile(r"(.+?)\s+out of\s+(\d+)$")

    def parse_value(val):
        if not isinstance(val, str):
            return (np.nan, np.nan)

        val = val.strip()
        match = pattern.match(val)

        if not match:
            return (np.nan, np.nan)

        raw_floor, max_floor = match.groups()
        max_floor = int(max_floor)

        raw_floor = raw_floor.strip()

        # Numeric floor
        if raw_floor.isdigit():
            return (int(raw_floor), max_floor)

        # Named floor
        if raw_floor in floor_map:
            return (floor_map[raw_floor], max_floor)

        return (np.nan, np.nan)

    df[[current_col, max_col]] = (
        df[source_col]
        .apply(parse_value)
        .apply(pd.Series)
    )

    return df

house = add_floor_columns(house, source_col='Floor')

# rewriting the four NaN floors. 'max_floor' is estimated using the mean ~0.5
house.loc[2553, 'current_floor'] = 3
house.loc[2883, 'current_floor'] = 1
house.loc[4490, 'current_floor'] = 1
house.loc[4560, 'current_floor'] = 1

house.loc[2553, 'max_floor'] = 6
house.loc[2883, 'max_floor'] = 2
house.loc[4490, 'max_floor'] = 2
house.loc[4560, 'max_floor'] = 2

house['floor_ratio'] = house['current_floor'] / house['max_floor']
#price_skew = house["Rent"].skew()
#print(f"Rent skewness: {price_skew:.2f} (right-skew suggests log-transforming the target)")
for column in house.select_dtypes(include='number').columns:
    print(f"Skewness in {column} is {house[column].skew():.2f}.")
print("DataFrame head:\n",house.head())
print(house[house['floor_ratio'].isna()])
print()
print(f"floor_ratio mean: {house['floor_ratio'].mean():.3f}")


In [ ]:
# transforming Rent
house['log_Rent'] = np.log(house['Rent'])


house['Furnishing_Status'] = house['Furnishing Status']
house['Area_Type'] = house['Area Type']

# Rent distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.histplot(house["Rent"], bins=40, kde=True, color="steelblue", ax=axes[0])
axes[0].set_title("Rent distribution (linear scale)")
axes[0].set_xlabel("Rent")
axes[0].set_ylabel("Count")

sns.histplot(np.log1p(house["Rent"]), bins=40, kde=True, color="darkorange", ax=axes[1])
axes[1].set_title("Rent distribution (log1p scale)")
axes[1].set_xlabel("log(1 + Rent)")
axes[1].set_ylabel("Count")

for column in house.select_dtypes(include='number').columns:
    print(f"Skewness in {column} is {house[column].skew():.2f}.")

In this case, log-transforming did not help with some skewness

In [ ]:
sns.pairplot(data=house)




---

After log transforming the data, there are some noticable linear trends.




In [ ]:
house.head()

---

Initial model with a very bad result:



In [ ]:
model = smf.ols("Rent ~ BHK + Size + current_floor + max_floor + C(Furnishing_Status) + Bathroom", data=house).fit()
print(model.summary())

Since BHK is the number of bedrooms, halls and kitchens, it is very likely that there is going to be some multicollinearity between it and Size. We will therefore keep only the size. At the same time, the 'max_floor' and 'current_floor' are somewhat related as well as 'current_floor' never exceeds 'max_floor'. For this reason, we will omit the 'max_floor'.

At the same time, we will use log-scaled variants of the feature

In [ ]:
# Main effects
NUMERIC_FEATURES = [
    "Size",
    "floor_ratio",
    "Bathroom"
]

CATEGORICAL_FEATURES = [
    "City",
    "Furnishing_Status",
    "Area_Type"
]

# Interactions
INTERACTIONS = [
    "Size:C(City)",
    "C(Furnishing_Status):C(City)",
    "Size:Bathroom"
]

# Main effects part
main_effects = (
    NUMERIC_FEATURES
    + [f"C({c})" for c in CATEGORICAL_FEATURES]
)

# Full formula
formula = "log_Rent ~ " + " + ".join(main_effects + INTERACTIONS)

model = smf.ols(formula, data=house).fit()
print(model.summary())

In [ ]:
# Dummy encode categorical variables
X = pd.get_dummies(
    house[NUMERIC_FEATURES],
    drop_first=True
)

# Add interaction terms manually
# Size × City
for col in X.filter(like="City_").columns:
    X[f"Size:{col}"] = house["Size"] * X[col]

# Furnishing × City
for f in X.filter(like="Furnishing_Status_").columns:
    for c in X.filter(like="City_").columns:
        X[f"{f}:{c}"] = X[f] * X[c]

# Size × Bathroom
X["Size:Bathroom"] = house["Size"] * house["Bathroom"]

# Add constant
X = sm.add_constant(X)

vif = pd.DataFrame({
    "feature": X.columns,
    "VIF": [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif.sort_values("VIF", ascending=False)

In [ ]:
# Main effects
NUMERIC_FEATURES = [
    "Size",
    "floor_ratio",
    "Bathroom"
]

CATEGORICAL_FEATURES = [
    "City",
    "Furnishing_Status",
    "Area_Type"
]

# Interactions
INTERACTIONS = [
    "Size:C(City)",
    "C(Furnishing_Status):C(City)",
]

# Main effects part
main_effects = (
    NUMERIC_FEATURES
    + [f"C({c})" for c in CATEGORICAL_FEATURES]
)

# Full formula
formula = "log_Rent ~ " + " + ".join(main_effects + INTERACTIONS)

model = smf.ols(formula, data=house).fit()
print(model.summary())

# Dummy encode categorical variables
X = pd.get_dummies(
    house[NUMERIC_FEATURES],
    drop_first=True
)

# Add interaction terms manually
# Size × City
for col in X.filter(like="City_").columns:
    X[f"Size:{col}"] = house["Size"] * X[col]

# Furnishing × City
for f in X.filter(like="Furnishing_Status_").columns:
    for c in X.filter(like="City_").columns:
        X[f"{f}:{c}"] = X[f] * X[c]


# Add constant
X = sm.add_constant(X)

vif = pd.DataFrame({
    "feature": X.columns,
    "VIF": [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif.sort_values("VIF", ascending=False)



---

VIFs indicate little multicollinearity present in the numeric part of the model.

---



In [ ]:
# centering continuus variables to reduce multicollinearity
house['Size_c'] = house['Size'] - house['Size'].mean()
house['Bathroom_c'] = house['Bathroom'] - house['Bathroom'].mean()

# Main effects
NUMERIC_FEATURES = [
    "Size_c",
    "floor_ratio",
    "Bathroom_c"
]

CATEGORICAL_FEATURES = [
    "City",
    "Furnishing_Status",
    "Area_Type"
]

# Interactions
INTERACTIONS = [

]
# Main effects part
main_effects = (
    NUMERIC_FEATURES
    + [f"C({c})" for c in CATEGORICAL_FEATURES]
)


# Full formula
formula = "log_Rent ~ " + " + ".join(main_effects + INTERACTIONS)

model = smf.ols(formula, data=house).fit()
print(model.summary())

model_initial_feature_engineering = model

# Ensure numeric
X_numeric = X.astype(float)
vif = pd.DataFrame({
    "feature": X_numeric.columns,
    "VIF": [variance_inflation_factor(X_numeric.values, i) for i in range(X_numeric.shape[1])]
})
vif

This current model has nice R^2 but has trong multicollinearity problems. Residual analysis follows.

In [ ]:
# Baseline influence tables
FEATURES = [
    "Size_c",
    "floor_ratio",
    "Bathroom_c",
    "City",
    "Furnishing_Status",
    "Area_Type",
]

def make_X(df, features=FEATURES):
    """Return design matrix with intercept."""
    return sm.add_constant(df[features], has_constant='add')

# Get influence measures
infl_base = model_initial_feature_engineering.get_influence()
sf_base = infl_base.summary_frame()

# Thresholds
p_base = make_X(house).shape[1] - 1
n_base = len(house)
cook_thr_base = 4 / n_base
lev_thr_base = 2 * (p_base + 1) / n_base

# Count how many thresholds each point exceeds
thresholds_exceeded = (
    (sf_base['cooks_d'] > cook_thr_base).astype(int) +
    (sf_base['hat_diag'] > lev_thr_base).astype(int) +
    (sf_base['student_resid'].abs() > 3).astype(int) +
    (sf_base['cooks_d'] > 0.5).astype(int)
)

# Flag points that exceed at least 2 thresholds
flag_mask_base = thresholds_exceeded >= 2
flagged_indices_base = house.index[flag_mask_base]

print(f"Number of points flagged: {flag_mask_base.sum()}")

print(f"Baseline: {flag_mask_base.sum()} influential points flagged "
      f"(Cook>{cook_thr_base:.4f} or leverage>{lev_thr_base:.4f} or |studentized|>3)")

# Rank top 20 by different influence metrics
ranked = {
    'DFFITS': sf_base['dffits'].abs().nlargest(20),
    'Max|DFBETA|': pd.DataFrame(infl_base.dfbetas, columns=model_initial_feature_engineering.params.index).abs().max(axis=1).nlargest(20),
    'Leverage': sf_base['hat_diag'].nlargest(20),
    'CooksD': sf_base['cooks_d'].nlargest(20)
}

for key, series in ranked.items():
    tbl = pd.DataFrame({
        'row_index': series.index,  # using row indices instead of 'id'
        key: series.values
    })
    print(f"Baseline top 20 by {key}:")
    display(tbl)




In [ ]:
# Diagnostics helper
def plot_diagnostics(model, sf_local, label, ax_row, cook_thr_val):
    fitted = model.fittedvalues
    resid = model.resid

    # Residuals vs Fitted
    ax_row[0].scatter(fitted, resid, alpha=0.5)
    ax_row[0].axhline(0, color='red', linestyle='--')
    ax_row[0].set_title(f'{label}: Residuals vs Fitted')
    ax_row[0].set_xlabel('Fitted log-price')
    ax_row[0].set_ylabel('Residuals')

    # QQ plot
    sm.qqplot(resid, ax=ax_row[1], line='s')
    ax_row[1].set_title(f'{label}: QQ plot')

    # Cook's distance
    ax_row[2].stem(np.arange(len(sf_local)), sf_local['cooks_d'].values, basefmt=' ')
    ax_row[2].axhline(cook_thr_val, color='orange', linestyle='--', label='Cook threshold')
    ax_row[2].set_title(f"{label}: Cook's distance")
    ax_row[2].legend()

# Baseline diagnostics
infl_base = model_initial_feature_engineering.get_influence()
sf_base = infl_base.summary_frame()
p_base = make_X(house).shape[1] - 1
n_base = len(house)
cook_thr_base = 4 / n_base

# Flag influential points
flag_mask_base = (
    (sf_base['cooks_d'] > cook_thr_base) |
    (sf_base['hat_diag'] > (2 * (p_base + 1) / n_base)) |
    (sf_base['student_resid'].abs() > 3) |
    (sf_base['cooks_d'] > 0.5)
)
flagged_indices_base = house.index[flag_mask_base]  # use row indices

# Plot diagnostics: Residuals, QQ, Cook's distance
fig, axes = plt.subplots(1, 3, figsize=(18, 5))  # 1 row, 3 columns
plot_diagnostics(model_initial_feature_engineering, sf_base, 'Baseline', axes, cook_thr_base)

plt.tight_layout()
plt.show()

print(f"Baseline flagged rows: {len(flagged_indices_base)} (examples: {flagged_indices_base.tolist()[:10]})")

# Additional diagnostic: Scale-Location plot
fig, ax = plt.subplots(figsize=(14, 5))  # single Axes object
ax.scatter(model_initial_feature_engineering.fittedvalues,
           np.sqrt(np.abs(sf_base['student_resid'])),
           alpha=0.5)
ax.set_xlabel('Fitted values')
ax.set_ylabel('√|Standardized residuals|')
ax.set_title('Baseline: Scale-Location Plot')

plt.tight_layout()
plt.show()

print("""
Diagnostic Summary:
- Residuals vs Fitted: Check for non-linearity and heteroscedasticity
- QQ Plot: Check normality of residuals (deviations in tails indicate heavy-tailed errors)
- Cook's Distance: Identifies influential observations
- Scale-Location: Check for heteroscedasticity (spread should be constant)

Observations:
- Residuals vs Fitted: mostly linear and heteroscedastic, outliers can be seen
- Q-Q Plot: Indicates extremal values, the middle is very nicely normal
- Cooks distance: there are a few very distant observations
- Scale-Location: Suggests good heteroscedaticity, shows outliers
""")

In [ ]:
# -------------------------------
# Step 0: Reset index for filtered dataframe
# -------------------------------
thresholds_exceeded = (
    (sf_base['cooks_d'] > cook_thr_base).astype(int) +
    (sf_base['hat_diag'] > (2 * (p_base + 1) / n_base)).astype(int) +
    (sf_base['student_resid'].abs() > 3).astype(int) +
    (sf_base['cooks_d'] > 0.5).astype(int)
)
flag_mask_multi = thresholds_exceeded >= 2
flagged_indices_multi = house.index[flag_mask_multi]

houses_influentials_removed = house.drop(index=flagged_indices_multi).reset_index(drop=True)

# -------------------------------
# Step 1: Define features
# -------------------------------
numeric_features = ['Size_c', 'floor_ratio', 'Bathroom_c']
categorical_features = ['City', 'Furnishing_Status', 'Area_Type']

# -------------------------------
# Step 2: One-hot encode categoricals after filtering
# -------------------------------
X_filtered = pd.get_dummies(
    houses_influentials_removed[numeric_features + categorical_features],
    drop_first=True
)

# Ensure all numeric
X_filtered = X_filtered.astype(float)

# Add constant for intercept
X_filtered = sm.add_constant(X_filtered, has_constant='add')

# -------------------------------
# Step 3: Target variable
# -------------------------------
y_filtered = houses_influentials_removed['log_Rent'].astype(float)

# -------------------------------
# Step 4: Fit OLS
# -------------------------------
model_filtered = sm.OLS(y_filtered, X_filtered).fit()

# -------------------------------
# Step 5: Influence diagnostics
# -------------------------------
infl_filtered = model_filtered.get_influence()
sf_filtered = infl_filtered.summary_frame()
n_filtered = len(houses_influentials_removed)
cook_thr_filtered = 4 / n_filtered

# -------------------------------
# Step 6: Plot diagnostics
# -------------------------------
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
plot_diagnostics(model_filtered, sf_filtered, 'Filtered', axes, cook_thr_filtered)
plt.tight_layout()
plt.show()

# Scale-Location plot
fig, ax = plt.subplots(figsize=(14, 5))
ax.scatter(model_filtered.fittedvalues,
           np.sqrt(np.abs(sf_filtered['student_resid'])),
           alpha=0.5)
ax.set_xlabel('Fitted values')
ax.set_ylabel('√|Standardized residuals|')
ax.set_title('Filtered: Scale-Location Plot')
plt.tight_layout()
plt.show()

print(f"""
Filtered Diagnostic Summary:
- Number of observations removed: {flag_mask_multi.sum()}
- Shape of filtered dataset: {houses_influentials_removed.shape}
- Number of features used in model: {X_filtered.shape[1]}
""")

In [ ]:
print(model_filtered.summary())

In [ ]:
# X_filtered should already have constant and all numeric columns
vif_df = pd.DataFrame({
    "feature": X_filtered.columns,
    "VIF": [variance_inflation_factor(X_filtered.values, i) for i in range(X_filtered.shape[1])]
})

# Sort by VIF descending
vif_df = vif_df.sort_values("VIF", ascending=False).reset_index(drop=True)
print(vif_df)


## Questions for a linear regression analysis of house rent

When building a linear regression model for rent, it is useful to think
in terms of a workflow:

1. **Understand the data**
   - What is the response variable (e.g. `Rent`)?  
     What are the main predictor types (numeric, categorical, locations,
     amenities)?
   - Are there obvious data quality issues (missing values, impossible
     values, outliers)?

2. **Preprocessing and feature engineering**
   - How should categorical variables (e.g. city, furnishing status,
     point of contact) be encoded for a linear model (one?hot encoding,
     target encoding, etc.)?
   - Which numeric variables might benefit from scaling (standardization
     or robust scaling), and why can this matter for regularized
     regression?
   - Are there interactions that are conceptually meaningful
     (e.g. `BHK , Size`, `City , DistanceFromMainArea`)?
   - Can we create more interpretable features (e.g. rent per square
     foot, distance to city centre bins)?

3. **Transformations of response and regressors**
   - Is the distribution of `Rent` highly skewed or heavy tailed? Would a
     log transformation (modeling $\log(\text{Rent})$) stabilize
     variance and make residuals closer to normal?
   - Do some predictors show non linear relationships with rent? Would
     polynomial terms, splines, or monotone transforms (log, square
     root) be appropriate?
   - Are there predictors that should be centered or standardized before
     creating interaction or polynomial terms?

4. **Model specification and selection**
   - Start with a simple baseline: which variables should be included in
     a first OLS model, and how do residual plots look?
   - How to compare alternative specifications
     (different sets of features, transformed vs untransformed variables)
     using cross validation or a validation set?
   - When is it useful to move from plain OLS to regularized models such
     as ridge or lasso (e.g. many correlated predictors, high variance)?

5. **Model evaluation and diagnostics**
   - How to check linear model assumptions: residual vs fitted plots,
     QQ plots, heteroscedasticity, influential observations?
   - Which error metrics are most relevant here
     (RMSE, MAE, MAPE)?  How do training and test errors compare
     (overfitting vs underfitting)?
   - Are there systematic groups of houses (by city, BHK, furnishing)
     for which the model performs much worse, suggesting missing
     structure or interactions?



### More detailed questions to explore

- **Preprocessing**
  - How should missing values be handled for each variable (impute,
    drop, or create explicit missing indicators)?
  - Do we need to cap or Winsorize extreme values of `Rent` or `Size`
    before fitting a linear model?
  - Are there rare categories (e.g. cities or furnishing statuses with
    very few observations) that should be grouped together?

- **Transformations and linearity**
  - Plot `Rent` (or $\log(\text{Rent})$) against key predictors:
    `Size`, `BHK`, `Bathroom`, `City`, etc.  Do the relationships look
    approximately linear after transformation?
  - Would modeling $\log(\text{Rent})$ make residuals more symmetric and
    reduce heteroscedasticity?

- **Multicollinearity and regularization**
  - Are some predictors strongly correlated (e.g. `Size` and `BHK`)?  How
    do VIFs and condition numbers look for the chosen design matrix?
  - How do ridge and lasso behave in this dataset in terms of coefficient
    shrinkage and variable selection?
  - Which predictors consistently get selected by lasso across
    cross?validation folds?

- **Model selection and validation**
  - How does test error change when we:
    1. Add more predictors,
    2. Add interaction terms,
    3. Add polynomial terms,
    4. Switch from OLS to ridge/lasso?
  - How to choose the final model: by minimum cross?validated RMSE,
    parsimony (fewest predictors), or domain interpretability?

Use these questions as a checklist to design your own modeling pipeline
for the house rent dataset using linear regression and its regularized
variants.




---


There are no NaNs. There seem to be no immediate outliers.